In [20]:
from rdkit import Chem
from rdkit.Chem import Descriptors
import pubchempy as pcp
import pandas as pd

# Wczytanie pliku Excel
file_path = 'smiles.xlsx'  # Zmień na ścieżkę do pliku Excel
df = pd.read_excel(file_path)

# Zakładając, że kolumna ze SMILES ma nazwę 'SMILES', można ją wyodrębnić
smiles_list = df['SMILES'].tolist()

# Funkcja do pobrania numeru CAS
def get_cas(smiles):
    try:
        results = pcp.get_compounds(smiles, 'smiles')
        if results:
            for prop in results[0].synonyms:
                if '-' in prop and prop[0].isdigit():  # Filtrujemy CAS (format XX-XX-X)
                    return prop
        return "Błąd przy pobieraniu CAS"  # Jeśli nie uda się znaleźć CAS
    except Exception as e:
        return f"Błąd przy pobieraniu CAS: {str(e)}"  # Zwracamy błąd, jeśli wystąpi

# Generowanie danych
data = []
for smiles in smiles_list:
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        try:
            mass = Descriptors.MolWt(mol)  # Masa molowa
        except Exception as e:
            mass = f"Błąd przy obliczaniu masy: {str(e)}"  # Błąd przy obliczaniu masy
        cas = get_cas(smiles)  # Numer CAS
        data.append({"SMILES": smiles, "MW (g/mol)": mass, "CASRN": cas})
    else:
        data.append({"SMILES": smiles, "MW (g/mol)": "Błąd przy przetwarzaniu SMILES", "CASRN": "Błąd przy przetwarzaniu SMILES"})

# Konwersja do DataFrame
df_result = pd.DataFrame(data)

# Zapisanie wyników do pliku Excel (możesz dodać swoją ścieżkę)
df_result.to_excel('output_smiles.xlsx', index=False)

print(df_result)


[11:50:16] SMILES Parse Error: syntax error while parsing: UVB
[11:50:16] SMILES Parse Error: Failed parsing SMILES 'UVB' for input: 'UVB'
[11:50:16] SMILES Parse Error: syntax error while parsing: ion
[11:50:16] SMILES Parse Error: Failed parsing SMILES 'ion' for input: 'ion'


                                               SMILES MW (g/mol)  \
0                                                 O=C     30.026   
1                                      C1=CC=C(C=C1)O     94.113   
2                                      CC(=O)CCC(O)=O    116.116   
3                                         NCC1=CC=CO1     97.117   
4          CC(CCC(O)=O)(C1=CC=C(O)C=C1)C1=CC=C(O)C=C1    286.327   
..                                                ...        ...   
70                           CCCCCCCCCC[Si](OC)(OC)OC    262.466   
71                             CC(C)C[Si](CC(C)C)(O)O    176.332   
72  CO[Si](CCC(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F)...    468.282   
73           C(CCC[Si](OC)(OC)OC)(=O)/C=C/C1C=CC=CC=1    294.423   
74                               CCO[Si](OCC)(OCC)OCC     208.33   

                                                CASRN  
0                                             50-00-0  
1                                            108-95-2  
2              

In [21]:
# Zapisz do pliku Excel
output_file = 'smiles_data.xlsx'
df.to_excel(output_file, index=False)  # Zapisuje DataFrame do pliku Excel
print(f"Dane zapisano do pliku {output_file}")

Dane zapisano do pliku smiles_data.xlsx


In [1]:
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors
import pubchempy as pcp
import pandas as pd

# Wczytanie pliku Excel
file_path = 'smiles.xlsx'  # Podaj ścieżkę do pliku Excel
df = pd.read_excel(file_path)

# Zakładamy, że kolumna z SMILES nazywa się 'SMILES'
smiles_list = df['SMILES'].tolist()

# Funkcja do pobrania numeru CAS
def get_cas(smiles):
    try:
        results = pcp.get_compounds(smiles, 'smiles')
        if results:
            for prop in results[0].synonyms:
                if '-' in prop and prop[0].isdigit():  # Filtrujemy CAS (format XX-XX-X)
                    return prop
        return "Brak CAS w PubChem"
    except Exception as e:
        return f"Błąd CAS: {str(e)}"

# Funkcja do pobrania nazwy IUPAC z PubChem
def get_name_pubchem(smiles):
    try:
        results = pcp.get_compounds(smiles, 'smiles')
        if results:
            return results[0].iupac_name  # Pobranie nazwy IUPAC
        return "Brak nazwy w PubChem"
    except Exception as e:
        return f"Błąd nazwy PubChem: {str(e)}"

# Funkcja do pobrania wzoru sumarycznego z RDKit
def get_formula_rdkit(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            return rdMolDescriptors.CalcMolFormula(mol)  # Wzór sumaryczny
        return "Błąd przetwarzania"
    except Exception as e:
        return f"Błąd RDKit: {str(e)}"

# Generowanie danych
data = []
for smiles in smiles_list:
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        try:
            mass = Descriptors.MolWt(mol)  # Masa molowa
        except Exception as e:
            mass = f"Błąd masy: {str(e)}"

        cas = get_cas(smiles)  # Numer CAS
        name_pubchem = get_name_pubchem(smiles)  # Nazwa IUPAC
        formula_rdkit = get_formula_rdkit(smiles)  # Wzór sumaryczny

        data.append({
            "SMILES": smiles,
            "MW (g/mol)": mass,
            "CASRN": cas,
            "Name (PubChem)": name_pubchem,
            "Formula (RDKit)": formula_rdkit
        })
    else:
        data.append({
            "SMILES": smiles,
            "MW (g/mol)": "Błąd SMILES",
            "CASRN": "Błąd SMILES",
            "Name (PubChem)": "Błąd SMILES",
            "Formula (RDKit)": "Błąd SMILES"
        })

# Konwersja do DataFrame
df_result = pd.DataFrame(data)

# Zapis do pliku Excel
output_file = 'output_smiles.xlsx'
df_result.to_excel(output_file, index=False)

print(f"Wyniki zapisano do: {output_file}")
print(df_result)


[12:19:15] SMILES Parse Error: syntax error while parsing: UVB
[12:19:15] SMILES Parse Error: Failed parsing SMILES 'UVB' for input: 'UVB'
[12:19:15] SMILES Parse Error: syntax error while parsing: ion
[12:19:15] SMILES Parse Error: Failed parsing SMILES 'ion' for input: 'ion'


Wyniki zapisano do: output_smiles.xlsx
                                               SMILES MW (g/mol)  \
0                                                 O=C     30.026   
1                                      C1=CC=C(C=C1)O     94.113   
2                                      CC(=O)CCC(O)=O    116.116   
3                                         NCC1=CC=CO1     97.117   
4          CC(CCC(O)=O)(C1=CC=C(O)C=C1)C1=CC=C(O)C=C1    286.327   
..                                                ...        ...   
70                           CCCCCCCCCC[Si](OC)(OC)OC    262.466   
71                             CC(C)C[Si](CC(C)C)(O)O    176.332   
72  CO[Si](CCC(C(C(C(C(C(F)(F)F)(F)F)(F)F)(F)F)(F)...    468.282   
73           C(CCC[Si](OC)(OC)OC)(=O)/C=C/C1C=CC=CC=1    294.423   
74                               CCO[Si](OCC)(OCC)OCC     208.33   

                                          CASRN  \
0                                       50-00-0   
1                                      108